In [1]:
import pandas as pd
import numpy as np
import cx_Oracle
import os
import json
import datetime
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings(action="ignore")
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import bs4
import requests
import time
import math
import random
from shapely.geometry import Point, Polygon
import time as timesys
from pyproj import Geod
from haversine import haversine
pd.set_option('display.max_columns', None)

In [2]:
the_number_of_segment = 20

# 전항지 출항 확인

전항지 출항 지점 확인

In [3]:
import os
 
path_dir = 'voy_path'
 
file_list = os.listdir(path_dir)
file_new_list = []
for i in file_list:
    if "portmis" in i:
        file_new_list.append(i)

In [4]:
# ATA 확인
def check_ATA(df_test):
    for i in df_test.index:
        LONGITUDE = df_test.loc[i, 'longitude']
        LATITUDE = df_test.loc[i, 'latitude']
        if Point(LONGITUDE, LATITUDE).within(poly_busan):    
            stop_index = i - 1
            break
            continue
    return stop_index

# 시각화 툴로 processed된 것들 확인

항로 클러스터링

In [5]:
# dense_upper_china_list = ['COSCO_FOS_VRKR6_23processed'
# ,'AKA_BHUM_9V7282_7processed'
# , 'COSCO_FOS_VRKR6_25processed'
# , 'MSC_LA_SPEZIA_DIGX2_2processed'
# , 'OOCL_NEW_YORK_VRJS2_2processed'
# , 'MSC_WASHINGTON_5LDP4_4processed'
# , 'MSC_SOFIA_PAZ_D5FL9_2processed'
# , 'ZHU_CHENG_XIN_ZHOU_VRKR6_26processed'
# , 'OOCL_HAMBURG_VRZK9_6processed'
# , 'SEAMAX_STRATFORD_V7EP2_6processed'
# , 'OOCL_LUXEMBOURG_VRGO3_5processed'
# , 'ZHU_CHENG_XIN_ZHOU_VRKR6_25processed'
# , 'MSC_LUCIANA_3FTY3_4processed'
# , 'MSC_RENEE_VRPL2_5processed'
# , 'COSCO_FOS_VRKR6_24processed'
# , 'COSCO_FOS_VRKR6_20processed'
# , 'COSCO_FOS_VRKR6_26processed'
# , 'ZHU_CHENG_XIN_ZHOU_VRKR6_23processed'
# , 'OOCL_HAMBURG_VRZK9_5processed']

# AKA_BHUM_voyages_list = [
#     'COSCO_FOS_VRKR6_2022_20processed', 'COSCO_FOS_VRKR6_2022_23processed',
#     'COSCO_FOS_VRKR6_2022_24processed', 'COSCO_FOS_VRKR6_2022_25processed',
#     'AKA_BHUM_9V7282_2022_7processed', 'COSCO_FOS_VRKR6_2022_27processed',
#     'COSCO_FOS_VRKR6_2022_28processed', 'AKA_BHUM_9V7282_2022_7processed',
#     'COSCO_FOS_VRKR6_2022_30processed'
# ]

AKA_BHUM_voyages_list = [
    'AKA_BHUM_9V7282_2022_7processed', 'AKA_BHUM_9V7282_2022_8processed'
]
OOCL_NEW_YORK_voyages_list = [
    'OOCL_NEW_YORK_VRJS2_2022_1processed',
    'OOCL_NEW_YORK_VRJS2_2022_2processed',
    'OOCL_NEW_YORK_VRJS2_2022_3processed'
]

OOCL_LUXEMBOURG_voyages_list = [
    'OOCL_LUXEMBOURG_VRGO3_2022_5processed',
    'OOCL_LUXEMBOURG_VRGO3_2022_6processed'
]

OOCL_GENOA_voyages_list = [
    'OOCL_GENOA_VROK9_2022_3processed', 'OOCL_GENOA_VROK9_2022_5processed'
]

OOCL_HAMBURG_voyages_list = [
    'OOCL_HAMBURG_VRZK9_2022_5processed', 'OOCL_HAMBURG_VRZK9_2022_6processed'
]

SEAMAX_STRATFORD_voyages_list = [
    'SEAMAX_STRATFORD_V7EP2_2022_6processed',
    'SEAMAX_STRATFORD_V7EP2_2022_7processed'
]

ZIM_CHARLESTORN_voyages_list = [
    'ZIM_CHARLESTON_VRUC3_2022_6processed',
    'ZIM_CHARLESTON_VRUC3_2022_7processed'
]

single_voyage_list = [
    'MSC_ARIANE_3FDA3_2022_1processed', 'MSC_DANIT_3FZU8_2022_2processed',
    'MSC_EVA_3EVM7_2022_1processed', 'MSC_EXPRESS_III_A8LG7_2022_1processed',
    'MSC_KANOKO_D5TM3_2023_1processed', 'MSC_LA_SPEZIA_DIGX2_2022_2processed',
    'MSC_VANESSA_HOWD_2023_1processed', 'MSC_VIRGO_CQEB9_2023_2processed',
    'MSC_SOFIA_PAZ_D5FL9_2022_2processed',
    'MSC_WASHINGTON_5LDP4_2022_4processed',
    'MSC_RAVENNA_A8ZU8_2022_3processed', 'MSC_VANDYA_3FDW4_2022_2processed',
    'MSC_RENEE_VRPL2_2022_5processed', 'SEASPAN_OCEANIA_VRBI2_2023_1processed',
    'SEASPAN_BREEZE_VRNL8_2023_1processed', 'VALENCE_9HA3398_2023_1processed'
]

In [6]:
upper_china_path_dir = 'voy_path/upper_china/processed/'

In [7]:
# 가는 길인데 굳이 오래 돌아갈 필요는 없잖아 알고리즘
def optimal_next_point(current_position, shortest_point_idx, shortest_distance,
                       path_df):
    posi_c = current_position
    latitude_i = path_df.loc[shortest_point_idx, "latitude"]
    longitude_i = path_df.loc[shortest_point_idx, "longitude"]
    posi_i = (latitude_i, longitude_i)
    min_d = shortest_distance
    n = shortest_point_idx + 1
    while True:
        try:
            latitude_n = path_df.loc[n, "latitude"]
            longitude_n = path_df.loc[n, "longitude"]
            posi_n = (latitude_n, longitude_n)
            distance = haversine(posi_c, posi_n, unit="km")
            if n == shortest_point_idx + 1:
                path_distance = haversine(posi_i, posi_n, unit="km")
                paths_distance = path_distance
            else:
                path_distance = haversine(posi_n, posi_previous_n, unit="km")
                paths_distance += path_distance

            if distance < (paths_distance + min_d):
                optimal_next_point_idx = n
                return optimal_next_point_idx
            elif n == path_df.index[-1]:
                print("no better optimal_point")
                return shortest_point_idx + 1
            else:
                n = n + 1
                posi_previous_n = posi_n
        except:
            print("no better optimal_point")
            return shortest_point_idx + 1

In [8]:
# 인덱스 번호를 가지고 항로의 segment 번호를 찾음
# 만약 segment의 마지막 번호라면 seg_no를 하나 더한다.

def find_segment_no(optimal_point_idx, path_df, the_number_of_segment):
    seg_no = optimal_point_idx // (len(path_df) / the_number_of_segment)
    next_no = (optimal_point_idx + 1) // (len(path_df) / the_number_of_segment)
    if seg_no != next_no:
        return int(next_no)
    else:
        return int(seg_no)

In [9]:
# df 내의 포인트 전체 길이 구하는 함수
def reckon_point_lengths(df_test_2):
    distance = 0
    df_test_2.reset_index(inplace=True, drop=True)
    for idx in df_test_2.index:
        if idx != 0:
            LONGITUDE = df_test_2.loc[idx, "longitude"]
            LATITUDE = df_test_2.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = df_test_2.loc[idx - 1, "longitude"]
            LATITUDE = df_test_2.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)

            distance += haversine(current_coord, previous_current_coord, unit="km")
    return distance

In [10]:
def cal_duration(speed, distance):
    hour = int(distance // km_h)
    minute = int(
        (distance / km_h - hour) * 60)
    second = ((distance / km_h - hour) * 60 - minute) * 60
    duration = datetime.timedelta(hours=hour,minutes=minute, seconds=second)
    return duration

# Tianjin to Busan PNIT

## AKA_BHUM

In [11]:
coordinates_array = pd.DataFrame(data={"latitude" : [],"longitude" : []})

for i in AKA_BHUM_voyages_list:
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    coordinates_array = coordinates_array.append(df[["latitude","longitude"]], ignore_index=True)

In [12]:
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
from shapely.geometry import LineString

In [14]:
from scipy import interpolate
import pandas as pd

## below 3 knot 제거

In [15]:
interpolation_number = "10000"

### interpolation 10000

In [16]:
max_speed_list = []
min_speed_list = []
for i in AKA_BHUM_voyages_list:
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    max_speed_list.append(max_speed)
    min_speed_list.append(min_speed)
min_speed = np.min(min_speed_list)
max_speed = np.max(max_speed_list)

# norm = plt.Normalize(min_speed, max_speed)
# cmap = plt.get_cmap('RdYlGn')
# sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
# Create a ScalarMappable with the colormap and normalization

mean_speed_dict = {"voyage" : [],"mean_speed" : []}

for i in AKA_BHUM_voyages_list:
#     fig, ax = plt.subplots()
    
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    # 3노트 이하 데이터 삭제
    data = data[data["speed"] >= 3]
    data.reset_index(inplace=True, drop=True)
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    vessel_name = data["vessel_name"].values[0]
    voyage_name = i.split('processed')[0]
    speed_std = data["speed"].std()
    
    
    
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data['timestamp_seconds'] = (data['timestamp'] - pd.Timestamp("1970-01-01")) / pd.Timedelta(seconds=1)

    # Create a function that performs the interpolation
    interp_longitude = interpolate.interp1d(data['timestamp_seconds'], data['longitude'])
    interp_latitude = interpolate.interp1d(data['timestamp_seconds'], data['latitude'])
    interp_speed = interpolate.interp1d(data['timestamp_seconds'], data['speed'])

    # Specify the range of new x values for the interpolation
    new_x = np.linspace(data['timestamp_seconds'].min(), data['timestamp_seconds'].max(), num=int(interpolation_number))

    # Use the interpolation function to estimate y values for the new x values
    interp_longitude_y = interp_longitude(new_x)
    interp_latitude_y = interp_latitude(new_x)
    interp_speed_y = interp_speed(new_x)
    # Create new DataFrame with the interpolated values
    interp_data = pd.DataFrame({'timestamp': new_x, 'longitude': interp_longitude_y, 'latitude': interp_latitude_y, 'speed': interp_speed_y})
#     i = i.split("processed")[0]
    interp_data.to_csv(upper_china_path_dir + 'interpolated/'+ f"interpolated_{i}_below_3knot_{interpolation_number}.csv", encoding="utf-8 sig", header=True, index=False)
    
    # plot the data
#     sc = ax.scatter(interp_data['longitude'], interp_data['latitude'],c = interp_data['speed'], cmap = cmap, norm = norm, s=0.1)

#     Add axis labels
#     ax.set_xlabel('Longitude')
#     ax.set_ylabel('Latitude')
#     ax.set_title(f"{vessel_name}_{voyage_name}")
#     # Add a colorbar
#     cbar = plt.colorbar(sc,label='Speed')

#         Show the plot
#     plt.show()
        
    check = data["speed"].apply(lambda x : True if x < 3 else False)
    s = data.drop(data[check].index)["speed"]
    print(f"보간법 전 일반 평균 속도 : {data['speed'].mean()}")
    print(f"보간법 전 일반 속도 표준편차: {data['speed'].std()}")
    print(f"보간법 전 저속 제거 평균 속도 : {s.mean()}")
    print(f'보간법 전 저속 제거 속도 표준편차: {s.std()}')
    print('-')
    check = interp_data["speed"].apply(lambda x : True if x < 3 else False)
    s = interp_data.drop(interp_data[check].index)["speed"]
    print(f"보간법 후 일반 평균 속도 : {interp_data['speed'].mean()}")
    print(f"보간법 후 일반 속도 표준편차: {interp_data['speed'].std()}")
    print(f"보간법 후 저속 제거 평균 속도 : {s.mean()}")
    mean_speed_dict['voyage'].append(voyage_name)
    mean_speed_dict['mean_speed'].append(s.mean())
    print(f'보간법 후 저속 제거 속도 표준편차: {s.std()}')

보간법 전 일반 평균 속도 : 9.372818311874106
보간법 전 일반 속도 표준편차: 3.188774981913939
보간법 전 저속 제거 평균 속도 : 9.372818311874106
보간법 전 저속 제거 속도 표준편차: 3.188774981913939
-
보간법 후 일반 평균 속도 : 10.043762945694306
보간법 후 일반 속도 표준편차: 2.483736978983716
보간법 후 저속 제거 평균 속도 : 10.043762945694306
보간법 후 저속 제거 속도 표준편차: 2.483736978983716
보간법 전 일반 평균 속도 : 8.860000000000001
보간법 전 일반 속도 표준편차: 1.4070373290383746
보간법 전 저속 제거 평균 속도 : 8.860000000000001
보간법 전 저속 제거 속도 표준편차: 1.4070373290383746
-
보간법 후 일반 평균 속도 : 8.973029394002777
보간법 후 일반 속도 표준편차: 2.2335932796100963
보간법 후 저속 제거 평균 속도 : 8.973029394002777
보간법 후 저속 제거 속도 표준편차: 2.2335932796100963


In [17]:
mean_speed_df = pd.DataFrame(mean_speed_dict)
list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage)

['AKA_BHUM_9V7282_2022_7', 'AKA_BHUM_9V7282_2022_8']

In [18]:
high_speed_path_data =[]
low_speed_path_data =[]
for i in AKA_BHUM_voyages_list:
    i = i.split("processed")[0]
    if i in list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage):
        i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
        df = pd.read_csv(upper_china_path_dir + i + ".csv")
        low_speed_path_data.append(df)
#     elif i in list(mean_speed_df[mean_speed_df["mean_speed"] >= 14].voyage):
#         i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
#         df = pd.read_csv(upper_china_path_dir + i + ".csv")
#         high_speed_path_data.append(df)

In [19]:
all_data = pd.concat(low_speed_path_data)

# Add a new index column representing the row number
all_data['index'] = range(len(all_data))
all_data["index"] = all_data.index
concaternated_df = all_data.groupby('index').mean()
concaternated_df.to_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv", encoding="utf-8 sig",header=True, index=False)

In [20]:
result_df_dict=  {
    "experiment_no" : [],
    "mean_speed" : [],
    "max_speed" : []
}

# ETA 실험 (interpolation 10000), AKA BHUM 7항차 저속

In [21]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.669615e+09,117.794500,38.979000,7.700000
1,1.669615e+09,117.795166,38.978525,7.779298
2,1.669615e+09,117.795832,38.978050,7.858596
3,1.669616e+09,117.796498,38.977575,7.937894
4,1.669616e+09,117.797164,38.977100,8.017193
...,...,...,...,...
9995,1.669881e+09,128.911219,34.852268,8.176007
9996,1.669881e+09,128.911352,34.853251,8.157005
9997,1.669881e+09,128.911484,34.854234,8.138003
9998,1.669882e+09,128.911617,34.855217,8.119002


In [22]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment 20

In [23]:
the_number_of_segment = 20

In [24]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [25]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [26]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

11.525651713483978
11.192940660157102
10.876914729531247
10.958090148166608
10.834673804919037
10.645070424339172
10.79549563464511
10.725746857629083
10.608701739952984
10.434357795428356
9.617239818190374
11.49665778734069
10.89260573738575
9.109980511538337
8.152094858612907
7.094969654638753
5.157841579210552
5.447664931972969
6.7130297644710115
7.888195245356821


### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [27]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [28]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_seg_{the_number_of_segment}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [29]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 03:58:42.341169
평균: 0 days 01:38:32.684934629


In [30]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
81,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.6065,38.2938,10.0,102.0,2022-11-03 05:20:40.775978,2022-11-02 20:16:25,2022-11-02 07:44:12,under way using engine,741.880266,1245.460598,2022-11-05 01:34:51.872955,2022-11-05 01:34:31,0 days 00:00:20.872955
262,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9255,34.2735,7.4,60.0,2022-11-05 01:31:02.428314,2022-11-04 16:30:34,2022-11-04 04:14:10,under way using engine,112.615917,205.608677,2022-11-05 01:35:32.154777,2022-11-05 01:34:31,0 days 00:01:01.154777
36,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.9483,38.6342,11.1,94.0,2022-11-02 22:01:34.353827,2022-11-02 13:00:34,2022-11-02 12:56:10,under way using engine,888.189640,1505.410268,2022-11-05 01:36:35.794979,2022-11-05 01:34:31,0 days 00:02:04.794979
60,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,120.8479,38.5566,11.5,91.0,2022-11-03 01:43:38.286928,2022-11-02 16:43:24,2022-11-02 04:38:10,under way using engine,814.007071,1391.457653,2022-11-05 01:37:17.731297,2022-11-05 01:34:31,0 days 00:02:46.731297
261,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9146,34.2690,7.3,59.0,2022-11-05 01:25:59.438198,2022-11-04 16:25:43,2022-11-04 04:14:10,under way using engine,113.686369,209.015615,2022-11-05 01:30:41.154777,2022-11-05 01:34:31,0 days 00:03:49.845223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4097,35.1087,12.4,145.0,2022-11-04 05:19:43.093566,2022-11-03 20:19:06,2022-11-02 07:44:12,under way using engine,310.800535,545.995500,2022-11-04 21:45:58.658831,2022-11-05 01:34:31,0 days 03:48:32.341169
140,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.5674,34.4923,16.5,175.0,2022-11-04 08:21:23.318549,2022-11-03 23:21:07,2022-11-02 07:44:12,under way using engine,301.872964,659.184072,2022-11-04 21:44:49.098228,2022-11-05 01:34:31,0 days 03:49:41.901772
150,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.9837,34.0632,8.4,88.0,2022-11-04 11:58:25.435480,2022-11-04 02:56:13,2022-11-02 07:44:12,under way using engine,278.663426,482.472950,2022-11-04 21:43:23.664678,2022-11-05 01:34:31,0 days 03:51:07.335322
103,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4000,35.1233,12.6,150.0,2022-11-04 05:14:38.376056,2022-11-03 20:14:16,2022-11-02 07:44:12,under way using engine,311.739382,551.733145,2022-11-04 21:41:08.658831,2022-11-05 01:34:31,0 days 03:53:22.341169


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [31]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [32]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_seg_{the_number_of_segment}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [33]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 04:01:07.590395
평균: 0 days 01:36:43.464540446


In [34]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
81,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.6065,38.2938,10.0,102.0,2022-11-03 05:20:40.775978,2022-11-02 20:16:25,2022-11-02 07:44:12,under way using engine,741.880266,1245.460598,2022-11-05 01:34:51.872955,2022-11-05 01:34:31,0 days 00:00:20.872955
235,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6274,34.1819,6.7,64.0,2022-11-04 23:14:46.584631,2022-11-04 14:14:22,2022-11-04 01:38:11,under way using engine,139.937034,270.161194,2022-11-05 01:35:31.409605,2022-11-05 01:34:31,0 days 00:01:00.409605
262,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9255,34.2735,7.4,60.0,2022-11-05 01:31:02.428314,2022-11-04 16:30:34,2022-11-04 04:14:10,under way using engine,112.615917,205.608677,2022-11-05 01:35:32.154777,2022-11-05 01:34:31,0 days 00:01:01.154777
180,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.0338,34.0775,6.5,79.0,2022-11-04 18:37:09.715818,2022-11-04 09:36:22,2022-11-02 07:44:12,under way using engine,191.309380,376.465514,2022-11-05 01:36:01.095888,2022-11-05 01:34:31,0 days 00:01:30.095888
36,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.9483,38.6342,11.1,94.0,2022-11-02 22:01:34.353827,2022-11-02 13:00:34,2022-11-02 12:56:10,under way using engine,888.189640,1505.410268,2022-11-05 01:36:35.794979,2022-11-05 01:34:31,0 days 00:02:04.794979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.1331,34.0882,6.4,76.0,2022-11-04 19:22:35.135832,2022-11-04 10:22:05,2022-11-02 07:44:12,under way using engine,182.943349,363.661714,2022-11-04 21:43:14.409605,2022-11-05 01:34:31,0 days 03:51:16.590395
103,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4000,35.1233,12.6,150.0,2022-11-04 05:14:38.376056,2022-11-03 20:14:16,2022-11-02 07:44:12,under way using engine,311.739382,551.733145,2022-11-04 21:41:08.658831,2022-11-05 01:34:31,0 days 03:53:22.341169
188,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.1193,34.0860,6.4,76.0,2022-11-04 19:17:32.215013,2022-11-04 10:15:34,2022-11-02 07:44:12,under way using engine,184.144127,366.048666,2022-11-04 21:36:43.409605,2022-11-05 01:34:31,0 days 03:57:47.590395
102,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.3902,35.1405,12.8,150.0,2022-11-04 05:09:35.685414,2022-11-03 20:08:56,2022-11-02 07:44:12,under way using engine,312.707561,557.826634,2022-11-04 21:35:48.658831,2022-11-05 01:34:31,0 days 03:58:42.341169


# ETA 실험 (interpolation 10000), AKA BHUM 7항차 저속

In [35]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.669615e+09,117.794500,38.979000,7.700000
1,1.669615e+09,117.795166,38.978525,7.779298
2,1.669615e+09,117.795832,38.978050,7.858596
3,1.669616e+09,117.796498,38.977575,7.937894
4,1.669616e+09,117.797164,38.977100,8.017193
...,...,...,...,...
9995,1.669881e+09,128.911219,34.852268,8.176007
9996,1.669881e+09,128.911352,34.853251,8.157005
9997,1.669881e+09,128.911484,34.854234,8.138003
9998,1.669882e+09,128.911617,34.855217,8.119002


In [36]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment 30

In [37]:
the_number_of_segment = 30

In [38]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [39]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [40]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

11.719029317195599
11.159365754893614
11.200244001760977
10.843635445710307
10.88929367435388
11.01648178121649
10.954074344637855
10.612814951916256
10.65762585195569
10.790408308554166
10.785088732290854
10.707136683918577
10.629184635546304
10.551232587174027
10.399764571897446
9.376555869939967
10.577899753495556
11.687660279813494
11.621001337174242
9.293349589361744
9.134176395297702
8.471749260550745
7.372617373550359
7.117813378437018
5.1038959136247195
5.315957833904582
5.479376522451545
6.181227819564282
7.8067379681327465
7.836211543265777


### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [41]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [42]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_seg_{the_number_of_segment}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [43]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 04:01:30.675181
평균: 0 days 01:26:21.585082521


In [44]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
12,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,118.5316,38.8073,12.2,98.0,2022-11-02 16:18:20.567192,2022-11-02 07:18:02,2022-11-02 07:14:14,under way using engine,1008.747013,1759.798302,2022-11-05 01:34:53.302080,2022-11-05 01:34:31,0 days 00:00:22.302080
236,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6385,34.1844,7.0,71.0,2022-11-04 23:19:48.866175,2022-11-04 14:19:22,2022-11-04 01:38:11,under way using engine,138.969866,261.416107,2022-11-05 01:35:20.262605,2022-11-05 01:34:31,0 days 00:00:49.262605
271,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,128.0404,34.3096,7.2,69.0,2022-11-05 02:21:30.779426,2022-11-04 17:20:54,2022-11-04 05:20:11,under way using engine,102.300361,189.459799,2022-11-05 01:35:23.705923,2022-11-05 01:34:31,0 days 00:00:52.705923
66,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.0790,38.4852,11.4,113.0,2022-11-03 02:44:12.568962,2022-11-02 17:43:53,2022-11-02 05:38:13,under way using engine,792.490261,1351.494352,2022-11-05 01:32:19.765228,2022-11-05 01:34:31,0 days 00:02:11.234772
270,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,128.0295,34.3068,7.2,70.0,2022-11-05 02:16:28.345945,2022-11-04 17:16:14,2022-11-04 05:14:10,under way using engine,103.220471,191.163838,2022-11-05 01:30:43.705923,2022-11-05 01:34:31,0 days 00:03:47.294077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6690,38.6594,11.8,105.0,2022-11-02 20:50:54.269771,2022-11-02 11:50:23,2022-11-02 08:14:16,under way using engine,911.455420,1570.416337,2022-11-04 22:05:00.324819,2022-11-05 01:34:31,0 days 03:29:30.675181
85,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.1918,35.4455,12.5,147.0,2022-11-04 03:33:41.281890,2022-11-03 18:32:16,2022-11-02 07:44:12,under way using engine,333.787423,588.528544,2022-11-04 21:58:47.766551,2022-11-05 01:34:31,0 days 03:35:43.233449
26,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6207,38.6709,11.7,124.0,2022-11-02 20:40:48.071161,2022-11-02 11:38:13,2022-11-02 08:14:16,under way using engine,915.807411,1573.536915,2022-11-04 21:52:50.324819,2022-11-05 01:34:31,0 days 03:41:40.675181
25,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6058,38.6752,12.0,101.0,2022-11-02 20:35:45.203472,2022-11-02 11:34:23,2022-11-02 08:14:16,under way using engine,917.183776,1589.732763,2022-11-04 21:49:00.324819,2022-11-05 01:34:31,0 days 03:45:30.675181


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [45]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [46]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_seg_{the_number_of_segment}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [47]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 04:01:30.675181
평균: 0 days 01:17:35.111568161


In [48]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
12,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,118.5316,38.8073,12.2,98.0,2022-11-02 16:18:20.567192,2022-11-02 07:18:02,2022-11-02 07:14:14,under way using engine,1008.747013,1759.798302,2022-11-05 01:34:53.302080,2022-11-05 01:34:31,0 days 00:00:22.302080
236,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6385,34.1844,7.0,71.0,2022-11-04 23:19:48.866175,2022-11-04 14:19:22,2022-11-04 01:38:11,under way using engine,138.969866,261.416107,2022-11-05 01:35:20.262605,2022-11-05 01:34:31,0 days 00:00:49.262605
271,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,128.0404,34.3096,7.2,69.0,2022-11-05 02:21:30.779426,2022-11-04 17:20:54,2022-11-04 05:20:11,under way using engine,102.300361,189.459799,2022-11-05 01:35:23.705923,2022-11-05 01:34:31,0 days 00:00:52.705923
254,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.8350,34.2406,6.9,57.0,2022-11-05 00:50:40.093837,2022-11-04 15:49:53,2022-11-04 03:32:10,under way using engine,121.196467,229.884165,2022-11-05 01:35:29.918400,2022-11-05 01:34:31,0 days 00:00:58.918400
66,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.0790,38.4852,11.4,113.0,2022-11-03 02:44:12.568962,2022-11-02 17:43:53,2022-11-02 05:38:13,under way using engine,792.490261,1351.494352,2022-11-05 01:32:19.765228,2022-11-05 01:34:31,0 days 00:02:11.234772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6690,38.6594,11.8,105.0,2022-11-02 20:50:54.269771,2022-11-02 11:50:23,2022-11-02 08:14:16,under way using engine,911.455420,1570.416337,2022-11-04 22:05:00.324819,2022-11-05 01:34:31,0 days 03:29:30.675181
85,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.1918,35.4455,12.5,147.0,2022-11-04 03:33:41.281890,2022-11-03 18:32:16,2022-11-02 07:44:12,under way using engine,333.787423,588.528544,2022-11-04 21:58:47.766551,2022-11-05 01:34:31,0 days 03:35:43.233449
26,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6207,38.6709,11.7,124.0,2022-11-02 20:40:48.071161,2022-11-02 11:38:13,2022-11-02 08:14:16,under way using engine,915.807411,1573.536915,2022-11-04 21:52:50.324819,2022-11-05 01:34:31,0 days 03:41:40.675181
25,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6058,38.6752,12.0,101.0,2022-11-02 20:35:45.203472,2022-11-02 11:34:23,2022-11-02 08:14:16,under way using engine,917.183776,1589.732763,2022-11-04 21:49:00.324819,2022-11-05 01:34:31,0 days 03:45:30.675181


# ETA 실험 (interpolation 10000), AKA BHUM 7항차 저속

In [49]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.669615e+09,117.794500,38.979000,7.700000
1,1.669615e+09,117.795166,38.978525,7.779298
2,1.669615e+09,117.795832,38.978050,7.858596
3,1.669616e+09,117.796498,38.977575,7.937894
4,1.669616e+09,117.797164,38.977100,8.017193
...,...,...,...,...
9995,1.669881e+09,128.911219,34.852268,8.176007
9996,1.669881e+09,128.911352,34.853251,8.157005
9997,1.669881e+09,128.911484,34.854234,8.138003
9998,1.669882e+09,128.911617,34.855217,8.119002


In [50]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment 40

In [51]:
the_number_of_segment = 40

In [52]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [53]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [54]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

10.825265715543177
12.226037711424775
11.195463318637852
11.190418001676354
10.853445723974563
10.90038373508793
10.737131977869797
11.179048318463417
11.04883964691418
10.620507962923897
10.620889251710636
10.66925159696771
10.77776398532849
10.813227283961734
10.755008137048113
10.696485578210053
10.637963019372007
10.579440460533961
10.520917901695917
10.347797689160794
9.34595683151131
9.888522804869435
11.304470715980163
11.688844858701216
11.715446790199683
10.06976468457182
9.000535247005176
9.2194257760715
8.633560446675766
7.670629270550049
7.08200925764469
7.107930051632817
5.076805441166002
5.238877717255102
5.4440218736192065
5.45130799032673
5.8939804047396915
7.532079124202333
8.022280583924339
7.754109906789303


### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [55]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [56]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_seg_{the_number_of_segment}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [57]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 02:48:36.719958
평균: 0 days 01:16:38.288633061


In [58]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
332,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,128.8250,34.6247,7.1,51.0,2022-11-05 08:50:08.153448,2022-11-04 23:48:20,2022-11-04 11:26:10,under way using engine,38.354531,71.576346,2022-11-05 01:34:37.473005,2022-11-05 01:34:31,0 days 00:00:06.473005
236,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6385,34.1844,7.0,71.0,2022-11-04 23:19:48.866175,2022-11-04 14:19:22,2022-11-04 01:38:11,under way using engine,138.969866,261.416107,2022-11-05 01:34:54.197077,2022-11-05 01:34:31,0 days 00:00:23.197077
263,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9361,34.2779,7.3,59.0,2022-11-05 01:36:05.224548,2022-11-04 16:35:23,2022-11-04 04:32:12,under way using engine,111.574020,205.132002,2022-11-05 01:35:00.047144,2022-11-05 01:34:31,0 days 00:00:29.047144
73,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.3444,38.3733,10.9,116.0,2022-11-03 03:59:55.023776,2022-11-02 18:59:25,2022-11-02 06:56:13,under way using engine,766.240503,1294.423396,2022-11-05 01:35:04.159042,2022-11-05 01:34:31,0 days 00:00:33.159042
333,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,128.8314,34.6292,7.0,42.0,2022-11-05 08:55:11.127437,2022-11-04 23:51:52,2022-11-04 11:26:10,under way using engine,37.866529,71.230699,2022-11-05 01:38:09.473005,2022-11-05 01:34:31,0 days 00:03:38.473005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,118.6534,38.7950,12.0,100.0,2022-11-02 16:58:43.005977,2022-11-02 07:55:19,2022-11-02 07:50:13,under way using engine,998.446536,1730.583565,2022-11-05 04:05:20.475048,2022-11-05 01:34:31,0 days 02:30:49.475048
102,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.3902,35.1405,12.8,150.0,2022-11-04 05:09:35.685414,2022-11-03 20:08:56,2022-11-02 07:44:12,under way using engine,312.707561,557.826634,2022-11-04 23:00:49.922482,2022-11-05 01:34:31,0 days 02:33:41.077518
5,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,118.1824,38.8602,19.6,129.0,2022-11-02 15:22:50.975897,2022-11-02 06:17:43,2022-11-02 06:20:14,under way using engine,1039.037228,2797.284621,2022-11-05 04:11:20.191127,2022-11-05 01:34:31,0 days 02:36:49.191127
19,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,118.6603,38.7941,12.0,101.0,2022-11-02 17:08:48.917276,2022-11-02 08:06:42,2022-11-02 07:50:13,under way using engine,997.854785,1729.557899,2022-11-05 04:16:43.475048,2022-11-05 01:34:31,0 days 02:42:12.475048


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [59]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [60]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_seg_{the_number_of_segment}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [61]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 02:48:36.719958
평균: 0 days 01:07:01.690251693


In [62]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
332,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,128.8250,34.6247,7.1,51.0,2022-11-05 08:50:08.153448,2022-11-04 23:48:20,2022-11-04 11:26:10,under way using engine,38.354531,71.576346,2022-11-05 01:34:37.473005,2022-11-05 01:34:31,0 days 00:00:06.473005
214,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.4051,34.1172,6.6,64.0,2022-11-04 21:28:46.251741,2022-11-04 12:28:24,2022-11-02 07:44:12,under way using engine,160.511091,312.798585,2022-11-05 01:34:21.689485,2022-11-05 01:34:31,0 days 00:00:09.310515
236,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6385,34.1844,7.0,71.0,2022-11-04 23:19:48.866175,2022-11-04 14:19:22,2022-11-04 01:38:11,under way using engine,138.969866,261.416107,2022-11-05 01:34:54.197077,2022-11-05 01:34:31,0 days 00:00:23.197077
263,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9361,34.2779,7.3,59.0,2022-11-05 01:36:05.224548,2022-11-04 16:35:23,2022-11-04 04:32:12,under way using engine,111.574020,205.132002,2022-11-05 01:35:00.047144,2022-11-05 01:34:31,0 days 00:00:29.047144
73,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.3444,38.3733,10.9,116.0,2022-11-03 03:59:55.023776,2022-11-02 18:59:25,2022-11-02 06:56:13,under way using engine,766.240503,1294.423396,2022-11-05 01:35:04.159042,2022-11-05 01:34:31,0 days 00:00:33.159042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,118.6534,38.7950,12.0,100.0,2022-11-02 16:58:43.005977,2022-11-02 07:55:19,2022-11-02 07:50:13,under way using engine,998.446536,1730.583565,2022-11-05 04:05:20.475048,2022-11-05 01:34:31,0 days 02:30:49.475048
102,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.3902,35.1405,12.8,150.0,2022-11-04 05:09:35.685414,2022-11-03 20:08:56,2022-11-02 07:44:12,under way using engine,312.707561,557.826634,2022-11-04 23:00:49.922482,2022-11-05 01:34:31,0 days 02:33:41.077518
5,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,118.1824,38.8602,19.6,129.0,2022-11-02 15:22:50.975897,2022-11-02 06:17:43,2022-11-02 06:20:14,under way using engine,1039.037228,2797.284621,2022-11-05 04:11:20.191127,2022-11-05 01:34:31,0 days 02:36:49.191127
19,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,118.6603,38.7941,12.0,101.0,2022-11-02 17:08:48.917276,2022-11-02 08:06:42,2022-11-02 07:50:13,under way using engine,997.854785,1729.557899,2022-11-05 04:16:43.475048,2022-11-05 01:34:31,0 days 02:42:12.475048


# ETA 실험 (interpolation 10000), AKA BHUM 7항차 저속

In [63]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.669615e+09,117.794500,38.979000,7.700000
1,1.669615e+09,117.795166,38.978525,7.779298
2,1.669615e+09,117.795832,38.978050,7.858596
3,1.669616e+09,117.796498,38.977575,7.937894
4,1.669616e+09,117.797164,38.977100,8.017193
...,...,...,...,...
9995,1.669881e+09,128.911219,34.852268,8.176007
9996,1.669881e+09,128.911352,34.853251,8.157005
9997,1.669881e+09,128.911484,34.854234,8.138003
9998,1.669882e+09,128.911617,34.855217,8.119002


In [64]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment 40

In [65]:
the_number_of_segment = 40

In [66]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [67]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [68]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

10.825265715543177
12.226037711424775
11.195463318637852
11.190418001676354
10.853445723974563
10.90038373508793
10.737131977869797
11.179048318463417
11.04883964691418
10.620507962923897
10.620889251710636
10.66925159696771
10.77776398532849
10.813227283961734
10.755008137048113
10.696485578210053
10.637963019372007
10.579440460533961
10.520917901695917
10.347797689160794
9.34595683151131
9.888522804869435
11.304470715980163
11.688844858701216
11.715446790199683
10.06976468457182
9.000535247005176
9.2194257760715
8.633560446675766
7.670629270550049
7.08200925764469
7.107930051632817
5.076805441166002
5.238877717255102
5.4440218736192065
5.45130799032673
5.8939804047396915
7.532079124202333
8.022280583924339
7.754109906789303


### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [69]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [70]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_seg_{the_number_of_segment}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [71]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 02:48:36.719958
평균: 0 days 01:16:38.288633061


In [72]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
332,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,128.8250,34.6247,7.1,51.0,2022-11-05 08:50:08.153448,2022-11-04 23:48:20,2022-11-04 11:26:10,under way using engine,38.354531,71.576346,2022-11-05 01:34:37.473005,2022-11-05 01:34:31,0 days 00:00:06.473005
236,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6385,34.1844,7.0,71.0,2022-11-04 23:19:48.866175,2022-11-04 14:19:22,2022-11-04 01:38:11,under way using engine,138.969866,261.416107,2022-11-05 01:34:54.197077,2022-11-05 01:34:31,0 days 00:00:23.197077
263,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9361,34.2779,7.3,59.0,2022-11-05 01:36:05.224548,2022-11-04 16:35:23,2022-11-04 04:32:12,under way using engine,111.574020,205.132002,2022-11-05 01:35:00.047144,2022-11-05 01:34:31,0 days 00:00:29.047144
73,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.3444,38.3733,10.9,116.0,2022-11-03 03:59:55.023776,2022-11-02 18:59:25,2022-11-02 06:56:13,under way using engine,766.240503,1294.423396,2022-11-05 01:35:04.159042,2022-11-05 01:34:31,0 days 00:00:33.159042
333,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,128.8314,34.6292,7.0,42.0,2022-11-05 08:55:11.127437,2022-11-04 23:51:52,2022-11-04 11:26:10,under way using engine,37.866529,71.230699,2022-11-05 01:38:09.473005,2022-11-05 01:34:31,0 days 00:03:38.473005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,118.6534,38.7950,12.0,100.0,2022-11-02 16:58:43.005977,2022-11-02 07:55:19,2022-11-02 07:50:13,under way using engine,998.446536,1730.583565,2022-11-05 04:05:20.475048,2022-11-05 01:34:31,0 days 02:30:49.475048
102,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.3902,35.1405,12.8,150.0,2022-11-04 05:09:35.685414,2022-11-03 20:08:56,2022-11-02 07:44:12,under way using engine,312.707561,557.826634,2022-11-04 23:00:49.922482,2022-11-05 01:34:31,0 days 02:33:41.077518
5,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,118.1824,38.8602,19.6,129.0,2022-11-02 15:22:50.975897,2022-11-02 06:17:43,2022-11-02 06:20:14,under way using engine,1039.037228,2797.284621,2022-11-05 04:11:20.191127,2022-11-05 01:34:31,0 days 02:36:49.191127
19,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,118.6603,38.7941,12.0,101.0,2022-11-02 17:08:48.917276,2022-11-02 08:06:42,2022-11-02 07:50:13,under way using engine,997.854785,1729.557899,2022-11-05 04:16:43.475048,2022-11-05 01:34:31,0 days 02:42:12.475048


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [73]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [74]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_seg_{the_number_of_segment}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [75]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 02:48:36.719958
평균: 0 days 01:07:01.690251693


In [76]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
332,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,128.8250,34.6247,7.1,51.0,2022-11-05 08:50:08.153448,2022-11-04 23:48:20,2022-11-04 11:26:10,under way using engine,38.354531,71.576346,2022-11-05 01:34:37.473005,2022-11-05 01:34:31,0 days 00:00:06.473005
214,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.4051,34.1172,6.6,64.0,2022-11-04 21:28:46.251741,2022-11-04 12:28:24,2022-11-02 07:44:12,under way using engine,160.511091,312.798585,2022-11-05 01:34:21.689485,2022-11-05 01:34:31,0 days 00:00:09.310515
236,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6385,34.1844,7.0,71.0,2022-11-04 23:19:48.866175,2022-11-04 14:19:22,2022-11-04 01:38:11,under way using engine,138.969866,261.416107,2022-11-05 01:34:54.197077,2022-11-05 01:34:31,0 days 00:00:23.197077
263,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9361,34.2779,7.3,59.0,2022-11-05 01:36:05.224548,2022-11-04 16:35:23,2022-11-04 04:32:12,under way using engine,111.574020,205.132002,2022-11-05 01:35:00.047144,2022-11-05 01:34:31,0 days 00:00:29.047144
73,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.3444,38.3733,10.9,116.0,2022-11-03 03:59:55.023776,2022-11-02 18:59:25,2022-11-02 06:56:13,under way using engine,766.240503,1294.423396,2022-11-05 01:35:04.159042,2022-11-05 01:34:31,0 days 00:00:33.159042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,118.6534,38.7950,12.0,100.0,2022-11-02 16:58:43.005977,2022-11-02 07:55:19,2022-11-02 07:50:13,under way using engine,998.446536,1730.583565,2022-11-05 04:05:20.475048,2022-11-05 01:34:31,0 days 02:30:49.475048
102,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.3902,35.1405,12.8,150.0,2022-11-04 05:09:35.685414,2022-11-03 20:08:56,2022-11-02 07:44:12,under way using engine,312.707561,557.826634,2022-11-04 23:00:49.922482,2022-11-05 01:34:31,0 days 02:33:41.077518
5,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,118.1824,38.8602,19.6,129.0,2022-11-02 15:22:50.975897,2022-11-02 06:17:43,2022-11-02 06:20:14,under way using engine,1039.037228,2797.284621,2022-11-05 04:11:20.191127,2022-11-05 01:34:31,0 days 02:36:49.191127
19,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,118.6603,38.7941,12.0,101.0,2022-11-02 17:08:48.917276,2022-11-02 08:06:42,2022-11-02 07:50:13,under way using engine,997.854785,1729.557899,2022-11-05 04:16:43.475048,2022-11-05 01:34:31,0 days 02:42:12.475048


# ETA 실험 (interpolation 10000), AKA BHUM 7항차 저속

In [77]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.669615e+09,117.794500,38.979000,7.700000
1,1.669615e+09,117.795166,38.978525,7.779298
2,1.669615e+09,117.795832,38.978050,7.858596
3,1.669616e+09,117.796498,38.977575,7.937894
4,1.669616e+09,117.797164,38.977100,8.017193
...,...,...,...,...
9995,1.669881e+09,128.911219,34.852268,8.176007
9996,1.669881e+09,128.911352,34.853251,8.157005
9997,1.669881e+09,128.911484,34.854234,8.138003
9998,1.669882e+09,128.911617,34.855217,8.119002


In [78]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment 50

In [79]:
the_number_of_segment = 50

In [80]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [81]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [82]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

9.874973639766756
13.404554443564678
11.09571528600105
11.231833100043778
11.18940446472644
10.879293001991739
10.820658959409565
10.945617254558712
10.688981684576714
11.252961293707909
11.12437322045212
10.657407852123578
10.61297327152004
10.625698664521538
10.678907564528252
10.76677993857289
10.823972183922502
10.784269416467142
10.73745136939668
10.69063332232627
10.643815275255802
10.596997228185401
10.550179181114922
10.50336113404453
10.313296019852697
9.34392334072313
9.534564076334949
10.604911230089789
11.625128824093633
11.676216542586157
11.148215720679952
11.401633856986164
9.24331077578826
8.991272120111008
9.222033148744842
8.746574986167241
7.900540122919808
7.381847446309446
7.008933340175624
7.079765387557027
5.053974476081084
5.217097230412196
5.282847678895301
5.52883378838503
5.431013104185186
5.567038757597311
7.307088646360071
7.881154223148976
7.943147269042654
7.804633628420572


### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [83]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [84]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_seg_{the_number_of_segment}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [85]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 03:13:24.529074
평균: 0 days 01:14:10.445769244


In [86]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
64,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.0239,38.5067,11.4,114.0,2022-11-03 02:29:03.864811,2022-11-02 17:28:44,2022-11-02 05:14:14,under way using engine,797.848008,1360.631328,2022-11-05 01:34:55.769183,2022-11-05 01:34:31,0 days 00:00:24.769183
258,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.8803,34.2556,7.2,62.0,2022-11-05 01:10:51.245788,2022-11-04 16:10:14,2022-11-04 03:32:10,under way using engine,117.003201,216.689391,2022-11-05 01:34:57.581109,2022-11-05 01:34:31,0 days 00:00:26.581109
55,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,120.6665,38.5762,11.3,91.0,2022-11-03 01:03:15.811867,2022-11-02 15:58:03,2022-11-02 04:02:10,under way using engine,829.079998,1410.781662,2022-11-05 01:33:30.151775,2022-11-05 01:34:31,0 days 00:01:00.848225
268,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,128.0046,34.3002,7.2,69.0,2022-11-05 02:06:22.186771,2022-11-04 17:05:33,2022-11-04 04:56:12,under way using engine,105.347326,195.102764,2022-11-05 01:36:11.309900,2022-11-05 01:34:31,0 days 00:01:40.309900
236,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6385,34.1844,7.0,71.0,2022-11-04 23:19:48.866175,2022-11-04 14:19:22,2022-11-04 01:38:11,under way using engine,138.969866,261.416107,2022-11-05 01:32:50.450531,2022-11-05 01:34:31,0 days 00:01:40.549469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6207,38.6709,11.7,124.0,2022-11-02 20:40:48.071161,2022-11-02 11:38:13,2022-11-02 08:14:16,under way using engine,915.807411,1573.536915,2022-11-04 22:40:56.470926,2022-11-05 01:34:31,0 days 02:53:34.529074
86,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.2264,35.3958,12.7,148.0,2022-11-04 03:48:49.927948,2022-11-03 18:48:26,2022-11-02 07:44:12,under way using engine,329.944606,586.228075,2022-11-04 22:39:40.071023,2022-11-05 01:34:31,0 days 02:54:50.928977
25,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6058,38.6752,12.0,101.0,2022-11-02 20:35:45.203472,2022-11-02 11:34:23,2022-11-02 08:14:16,under way using engine,917.183776,1589.732763,2022-11-04 22:37:06.470926,2022-11-05 01:34:31,0 days 02:57:24.529074
85,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.1918,35.4455,12.5,147.0,2022-11-04 03:33:41.281890,2022-11-03 18:32:16,2022-11-02 07:44:12,under way using engine,333.787423,588.528544,2022-11-04 22:23:30.071023,2022-11-05 01:34:31,0 days 03:11:00.928977


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [87]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [88]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_seg_{the_number_of_segment}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [89]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 03:13:24.529074
평균: 0 days 01:01:26.578008708


In [90]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
247,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.7599,34.2160,6.8,58.0,2022-11-05 00:15:19.951925,2022-11-04 15:15:04,2022-11-04 03:02:10,under way using engine,128.172002,245.228463,2022-11-05 01:34:13.196903,2022-11-05 01:34:31,0 days 00:00:17.803097
219,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.4571,34.1300,6.4,63.0,2022-11-04 21:54:01.215011,2022-11-04 12:53:24,2022-11-02 07:44:12,under way using engine,155.828380,309.761553,2022-11-05 01:34:10.005166,2022-11-05 01:34:31,0 days 00:00:20.994834
64,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.0239,38.5067,11.4,114.0,2022-11-03 02:29:03.864811,2022-11-02 17:28:44,2022-11-02 05:14:14,under way using engine,797.848008,1360.631328,2022-11-05 01:34:55.769183,2022-11-05 01:34:31,0 days 00:00:24.769183
177,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.0010,34.0743,6.5,79.0,2022-11-04 18:22:01.506034,2022-11-04 09:21:23,2022-11-02 07:44:12,under way using engine,194.069882,381.897731,2022-11-05 01:34:04.559060,2022-11-05 01:34:31,0 days 00:00:26.440940
258,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.8803,34.2556,7.2,62.0,2022-11-05 01:10:51.245788,2022-11-04 16:10:14,2022-11-04 03:32:10,under way using engine,117.003201,216.689391,2022-11-05 01:34:57.581109,2022-11-05 01:34:31,0 days 00:00:26.581109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6207,38.6709,11.7,124.0,2022-11-02 20:40:48.071161,2022-11-02 11:38:13,2022-11-02 08:14:16,under way using engine,915.807411,1573.536915,2022-11-04 22:40:56.470926,2022-11-05 01:34:31,0 days 02:53:34.529074
86,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.2264,35.3958,12.7,148.0,2022-11-04 03:48:49.927948,2022-11-03 18:48:26,2022-11-02 07:44:12,under way using engine,329.944606,586.228075,2022-11-04 22:39:40.071023,2022-11-05 01:34:31,0 days 02:54:50.928977
25,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6058,38.6752,12.0,101.0,2022-11-02 20:35:45.203472,2022-11-02 11:34:23,2022-11-02 08:14:16,under way using engine,917.183776,1589.732763,2022-11-04 22:37:06.470926,2022-11-05 01:34:31,0 days 02:57:24.529074
85,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.1918,35.4455,12.5,147.0,2022-11-04 03:33:41.281890,2022-11-03 18:32:16,2022-11-02 07:44:12,under way using engine,333.787423,588.528544,2022-11-04 22:23:30.071023,2022-11-05 01:34:31,0 days 03:11:00.928977


# ETA 실험 (interpolation 10000), AKA BHUM 7항차 저속

In [91]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.669615e+09,117.794500,38.979000,7.700000
1,1.669615e+09,117.795166,38.978525,7.779298
2,1.669615e+09,117.795832,38.978050,7.858596
3,1.669616e+09,117.796498,38.977575,7.937894
4,1.669616e+09,117.797164,38.977100,8.017193
...,...,...,...,...
9995,1.669881e+09,128.911219,34.852268,8.176007
9996,1.669881e+09,128.911352,34.853251,8.157005
9997,1.669881e+09,128.911484,34.854234,8.138003
9998,1.669882e+09,128.911617,34.855217,8.119002


In [92]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment 75

In [93]:
the_number_of_segment = 75

In [94]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [95]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [96]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

9.986190734735835
12.246206669978312
12.690824163765662
11.079629208376932
11.19455995507427
11.21650400937818
11.187295202255001
11.11529671655194
10.808965048575269
10.785937426749543
10.919937026742968
10.942352483476826
10.63834190739333
10.980413773097037
11.268859701521107
11.245942052868584
10.842344544900273
10.621197398329944
10.610471821779875
10.618894424702571
10.627318377236776
10.66111340878421
10.719548537523897
10.77798366626359
10.827307100919633
10.809668207002858
10.778534205700977
10.747400204399172
10.716266203097334
10.685132201795454
10.653998200493644
10.62286419919181
10.591730197889932
10.560596196588127
10.529462195286289
10.498328193984412
10.467194192682603
9.852856107625108
9.282281225506607
9.394937390344236
9.904700378615892
10.70665162842048
11.52014358088125
11.719469444553548
11.699192902692982
11.063349050784337
12.34621926964178
10.628684889165664
9.370567522236891
9.056272101291182
8.963589642658242
9.323491928236612
9.055084556123191
8.69767622819

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [97]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [98]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_seg_{the_number_of_segment}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [99]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 02:53:11.846677
평균: 0 days 01:03:25.412288478


In [100]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
54,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,120.6412,38.5789,11.4,91.0,2022-11-03 00:53:12.295650,2022-11-02 15:51:47,2022-11-02 03:50:12,under way using engine,831.183558,1417.480995,2022-11-05 01:34:48.791251,2022-11-05 01:34:31,0 days 00:00:17.791251
259,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.8914,34.2597,7.2,59.0,2022-11-05 01:15:53.638989,2022-11-04 16:15:15,2022-11-04 04:14:10,under way using engine,115.946719,214.732791,2022-11-05 01:33:25.715532,2022-11-05 01:34:31,0 days 00:01:05.284468
265,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9701,34.2904,7.2,65.0,2022-11-05 01:51:13.784099,2022-11-04 16:50:33,2022-11-04 04:50:10,under way using engine,108.363965,200.689564,2022-11-05 01:32:57.645302,2022-11-05 01:34:31,0 days 00:01:33.354698
78,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.5000,38.3216,10.2,102.0,2022-11-03 04:45:20.379906,2022-11-02 19:45:04,2022-11-02 07:44:12,under way using engine,751.547921,1261.801822,2022-11-05 01:36:06.004072,2022-11-05 01:34:31,0 days 00:01:35.004072
72,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.2920,38.3950,11.1,111.0,2022-11-03 03:44:47.215152,2022-11-02 18:44:24,2022-11-02 06:32:12,under way using engine,771.405759,1307.470948,2022-11-05 01:32:29.450769,2022-11-05 01:34:31,0 days 00:02:01.549231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.2895,35.3015,12.8,149.0,2022-11-04 04:19:07.187767,2022-11-03 19:18:35,2022-11-02 07:44:12,under way using engine,323.091153,576.349513,2022-11-04 23:01:28.153323,2022-11-05 01:34:31,0 days 02:33:02.846677
91,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.2795,35.3166,12.7,148.0,2022-11-04 04:14:08.339082,2022-11-03 19:13:46,2022-11-02 07:44:12,under way using engine,324.160217,575.950679,2022-11-04 22:56:39.153323,2022-11-05 01:34:31,0 days 02:37:51.846677
90,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.2690,35.3321,12.7,148.0,2022-11-04 04:09:01.581785,2022-11-03 19:08:47,2022-11-02 07:44:12,under way using engine,325.286452,577.951714,2022-11-04 22:51:40.153323,2022-11-05 01:34:31,0 days 02:42:50.846677
89,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.2575,35.3495,12.8,148.0,2022-11-04 04:04:01.408545,2022-11-03 19:03:16,2022-11-02 07:44:12,under way using engine,326.533959,582.490997,2022-11-04 22:46:09.153323,2022-11-05 01:34:31,0 days 02:48:21.846677


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [101]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [102]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_seg_{the_number_of_segment}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [103]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 02:53:11.846677
평균: 0 days 00:51:20.246042787


In [104]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
54,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,120.6412,38.5789,11.4,91.0,2022-11-03 00:53:12.295650,2022-11-02 15:51:47,2022-11-02 03:50:12,under way using engine,831.183558,1417.480995,2022-11-05 01:34:48.791251,2022-11-05 01:34:31,0 days 00:00:17.791251
252,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.8129,34.2334,6.8,57.0,2022-11-05 00:40:33.969379,2022-11-04 15:39:44,2022-11-04 03:32:10,under way using engine,123.240850,235.793806,2022-11-05 01:34:01.318912,2022-11-05 01:34:31,0 days 00:00:29.681088
214,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.4051,34.1172,6.6,64.0,2022-11-04 21:28:46.251741,2022-11-04 12:28:24,2022-11-02 07:44:12,under way using engine,160.511091,312.798585,2022-11-05 01:35:04.159439,2022-11-05 01:34:31,0 days 00:00:33.159439
194,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.1870,34.0929,6.5,84.0,2022-11-04 19:47:49.440341,2022-11-04 10:46:53,2022-11-02 07:44:12,under way using engine,178.504423,351.267457,2022-11-05 01:33:34.515604,2022-11-05 01:34:31,0 days 00:00:56.484396
226,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.5313,34.1517,6.7,61.0,2022-11-04 22:29:23.365344,2022-11-04 13:28:54,2022-11-02 07:44:12,under way using engine,148.945343,287.552556,2022-11-05 01:35:32.380560,2022-11-05 01:34:31,0 days 00:01:01.380560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.2895,35.3015,12.8,149.0,2022-11-04 04:19:07.187767,2022-11-03 19:18:35,2022-11-02 07:44:12,under way using engine,323.091153,576.349513,2022-11-04 23:01:28.153323,2022-11-05 01:34:31,0 days 02:33:02.846677
91,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.2795,35.3166,12.7,148.0,2022-11-04 04:14:08.339082,2022-11-03 19:13:46,2022-11-02 07:44:12,under way using engine,324.160217,575.950679,2022-11-04 22:56:39.153323,2022-11-05 01:34:31,0 days 02:37:51.846677
90,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.2690,35.3321,12.7,148.0,2022-11-04 04:09:01.581785,2022-11-03 19:08:47,2022-11-02 07:44:12,under way using engine,325.286452,577.951714,2022-11-04 22:51:40.153323,2022-11-05 01:34:31,0 days 02:42:50.846677
89,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.2575,35.3495,12.8,148.0,2022-11-04 04:04:01.408545,2022-11-03 19:03:16,2022-11-02 07:44:12,under way using engine,326.533959,582.490997,2022-11-04 22:46:09.153323,2022-11-05 01:34:31,0 days 02:48:21.846677


# ETA 실험 (interpolation 10000), AKA BHUM 7항차 저속

In [105]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.669615e+09,117.794500,38.979000,7.700000
1,1.669615e+09,117.795166,38.978525,7.779298
2,1.669615e+09,117.795832,38.978050,7.858596
3,1.669616e+09,117.796498,38.977575,7.937894
4,1.669616e+09,117.797164,38.977100,8.017193
...,...,...,...,...
9995,1.669881e+09,128.911219,34.852268,8.176007
9996,1.669881e+09,128.911352,34.853251,8.157005
9997,1.669881e+09,128.911484,34.854234,8.138003
9998,1.669882e+09,128.911617,34.855217,8.119002


In [106]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment 90

In [107]:
the_number_of_segment = 90

In [108]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [109]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [110]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

10.00990372375522
10.518506884596608
14.62867734323497
11.166843905789763
11.082098327426943
11.229155031464138
11.221568804757977
11.185328173361638
11.193835027163313
10.964637161458597
10.771560817160996
10.794708358511327
10.900617957521412
11.012840407083353
10.754422658456871
10.610072869490994
11.11774087114231
11.321631603016167
11.27408028336868
10.924364860147378
10.663777890397501
10.605785561507087
10.612814951916349
10.619844342325333
10.626873732734714
10.64886274098981
10.697141082142538
10.745910249737234
10.794679417332102
10.830635258593166
10.811072748414917
10.785088732290857
10.75910471616679
10.733120700042674
10.70713668391856
10.681152667794498
10.655168651670438
10.629184635546268
10.603200619422203
10.577216603298142
10.55123258717403
10.52524857104991
10.49926455492585
10.473280538801784
10.226748621964697
9.439302471626075
9.239020208969581
9.451344929224245
9.878904761877603
10.542562373480553
11.312232125128515
11.67478588729698
11.743156621908325
11.64503

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [111]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [112]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_seg_{the_number_of_segment}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [113]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 03:11:12.976273
평균: 0 days 01:08:27.125804690


In [114]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
261,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9146,34.2690,7.3,59.0,2022-11-05 01:25:59.438198,2022-11-04 16:25:43,2022-11-04 04:14:10,under way using engine,113.686369,209.015615,2022-11-05 01:34:21.656520,2022-11-05 01:34:31,0 days 00:00:09.343480
266,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9826,34.2943,7.2,68.0,2022-11-05 01:56:16.844120,2022-11-04 16:56:02,2022-11-04 04:50:10,under way using engine,107.241399,198.610578,2022-11-05 01:34:57.887819,2022-11-05 01:34:31,0 days 00:00:26.887819
255,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.8462,34.2445,7.0,57.0,2022-11-05 00:55:42.298259,2022-11-04 15:55:03,2022-11-04 03:32:10,under way using engine,120.143414,226.001683,2022-11-05 01:33:51.771014,2022-11-05 01:34:31,0 days 00:00:39.228986
46,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,120.3005,38.6128,11.4,89.0,2022-11-02 23:32:24.559531,2022-11-02 14:28:27,2022-11-02 02:32:14,under way using engine,859.460778,1465.704305,2022-11-05 01:33:22.023727,2022-11-05 01:34:31,0 days 00:01:08.976273
335,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,128.8501,34.6460,7.0,40.0,2022-11-05 09:05:24.557202,2022-11-05 00:03:32,2022-11-04 11:56:11,under way using engine,36.091374,67.891455,2022-11-05 01:36:18.780611,2022-11-05 01:34:31,0 days 00:01:47.780611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.2264,35.3958,12.7,148.0,2022-11-04 03:48:49.927948,2022-11-03 18:48:26,2022-11-02 07:44:12,under way using engine,329.944606,586.228075,2022-11-04 22:51:21.759597,2022-11-05 01:34:31,0 days 02:43:09.240403
26,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6207,38.6709,11.7,124.0,2022-11-02 20:40:48.071161,2022-11-02 11:38:13,2022-11-02 08:14:16,under way using engine,915.807411,1573.536915,2022-11-04 22:43:08.023727,2022-11-05 01:34:31,0 days 02:51:22.976273
25,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6058,38.6752,12.0,101.0,2022-11-02 20:35:45.203472,2022-11-02 11:34:23,2022-11-02 08:14:16,under way using engine,917.183776,1589.732763,2022-11-04 22:39:18.023727,2022-11-05 01:34:31,0 days 02:55:12.976273
85,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.1918,35.4455,12.5,147.0,2022-11-04 03:33:41.281890,2022-11-03 18:32:16,2022-11-02 07:44:12,under way using engine,333.787423,588.528544,2022-11-04 22:35:11.759597,2022-11-05 01:34:31,0 days 02:59:19.240403


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [115]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [116]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_seg_{the_number_of_segment}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [117]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 03:11:12.976273
평균: 0 days 00:54:49.992278455


In [118]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
261,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9146,34.2690,7.3,59.0,2022-11-05 01:25:59.438198,2022-11-04 16:25:43,2022-11-04 04:14:10,under way using engine,113.686369,209.015615,2022-11-05 01:34:21.656520,2022-11-05 01:34:31,0 days 00:00:09.343480
266,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9826,34.2943,7.2,68.0,2022-11-05 01:56:16.844120,2022-11-04 16:56:02,2022-11-04 04:50:10,under way using engine,107.241399,198.610578,2022-11-05 01:34:57.887819,2022-11-05 01:34:31,0 days 00:00:26.887819
208,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.3410,34.1035,6.3,76.0,2022-11-04 20:58:30.521894,2022-11-04 11:58:02,2022-11-02 07:44:12,under way using engine,166.172571,333.809188,2022-11-05 01:33:53.684316,2022-11-05 01:34:31,0 days 00:00:37.315684
255,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.8462,34.2445,7.0,57.0,2022-11-05 00:55:42.298259,2022-11-04 15:55:03,2022-11-04 03:32:10,under way using engine,120.143414,226.001683,2022-11-05 01:33:51.771014,2022-11-05 01:34:31,0 days 00:00:39.228986
186,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.0970,34.0830,6.6,80.0,2022-11-04 19:07:26.763468,2022-11-04 10:05:13,2022-11-02 07:44:12,under way using engine,186.053929,362.575604,2022-11-05 01:33:41.039837,2022-11-05 01:34:31,0 days 00:00:49.960163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.2264,35.3958,12.7,148.0,2022-11-04 03:48:49.927948,2022-11-03 18:48:26,2022-11-02 07:44:12,under way using engine,329.944606,586.228075,2022-11-04 22:51:21.759597,2022-11-05 01:34:31,0 days 02:43:09.240403
26,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6207,38.6709,11.7,124.0,2022-11-02 20:40:48.071161,2022-11-02 11:38:13,2022-11-02 08:14:16,under way using engine,915.807411,1573.536915,2022-11-04 22:43:08.023727,2022-11-05 01:34:31,0 days 02:51:22.976273
25,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6058,38.6752,12.0,101.0,2022-11-02 20:35:45.203472,2022-11-02 11:34:23,2022-11-02 08:14:16,under way using engine,917.183776,1589.732763,2022-11-04 22:39:18.023727,2022-11-05 01:34:31,0 days 02:55:12.976273
85,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.1918,35.4455,12.5,147.0,2022-11-04 03:33:41.281890,2022-11-03 18:32:16,2022-11-02 07:44:12,under way using engine,333.787423,588.528544,2022-11-04 22:35:11.759597,2022-11-05 01:34:31,0 days 02:59:19.240403


# ETA 실험 (interpolation 10000), AKA BHUM 7항차 저속

In [119]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.669615e+09,117.794500,38.979000,7.700000
1,1.669615e+09,117.795166,38.978525,7.779298
2,1.669615e+09,117.795832,38.978050,7.858596
3,1.669616e+09,117.796498,38.977575,7.937894
4,1.669616e+09,117.797164,38.977100,8.017193
...,...,...,...,...
9995,1.669881e+09,128.911219,34.852268,8.176007
9996,1.669881e+09,128.911352,34.853251,8.157005
9997,1.669881e+09,128.911484,34.854234,8.138003
9998,1.669882e+09,128.911617,34.855217,8.119002


In [120]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment 100

In [121]:
the_number_of_segment = 100

In [122]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [123]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [124]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

9.978291327149359
9.771655952384153
14.976630655037024
11.832478232092333
11.069202400757021
11.122228171245084
11.256532062161996
11.207134137925552
11.185461990422182
11.193346939030693
10.975830991526559
10.782755012456917
10.772611820963066
10.868706097856066
10.984669724853623
10.906564784263804
10.65792151945065
10.720041849702774
11.155884083218682
11.35003850419713
11.292582237410079
10.956164203494161
10.711341608955856
10.603474095291304
10.609806879443793
10.616139663596286
10.622472447748846
10.628924881294228
10.656939471017065
10.700875658039436
10.744811845061811
10.788748032083975
10.828561416075042
10.819382951769962
10.795973928234769
10.772564904699514
10.74915588116428
10.725746857629083
10.702337834093887
10.67892881055865
10.655519787023398
10.632110763488203
10.608701739953007
10.585292716417793
10.561883692882521
10.538474669347325
10.515065645812129
10.491656622276933
10.46824759874164
10.158344440963752
9.44295392574346
9.244892755702798
9.38768662869041
9.681

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [125]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [126]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_seg_{the_number_of_segment}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [127]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 02:41:00.964964
평균: 0 days 01:05:32.943752793


In [128]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
264,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9472,34.2824,7.2,59.0,2022-11-05 01:41:07.975263,2022-11-04 16:40:24,2022-11-04 04:32:12,under way using engine,110.492280,204.631195,2022-11-05 01:35:25.121648,2022-11-05 01:34:31,0 days 00:00:54.121648
268,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,128.0046,34.3002,7.2,69.0,2022-11-05 02:06:22.186771,2022-11-04 17:05:33,2022-11-04 04:56:12,under way using engine,105.347326,195.102764,2022-11-05 01:33:36.497007,2022-11-05 01:34:31,0 days 00:00:54.502993
258,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.8803,34.2556,7.2,62.0,2022-11-05 01:10:51.245788,2022-11-04 16:10:14,2022-11-04 03:32:10,under way using engine,117.003201,216.689391,2022-11-05 01:32:08.148302,2022-11-05 01:34:31,0 days 00:02:22.851698
259,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.8914,34.2597,7.2,59.0,2022-11-05 01:15:53.638989,2022-11-04 16:15:15,2022-11-04 04:14:10,under way using engine,115.946719,214.732791,2022-11-05 01:37:09.148302,2022-11-05 01:34:31,0 days 00:02:38.148302
263,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9361,34.2779,7.3,59.0,2022-11-05 01:36:05.224548,2022-11-04 16:35:23,2022-11-04 04:32:12,under way using engine,111.574020,205.132002,2022-11-05 01:30:24.121648,2022-11-05 01:34:31,0 days 00:04:06.878352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,122.7751,38.0019,11.1,97.0,2022-11-03 12:14:32.908857,2022-11-03 02:58:16,2022-11-02 07:44:12,under way using engine,636.722525,1079.193658,2022-11-05 03:55:30.660447,2022-11-05 01:34:31,0 days 02:20:59.660447
26,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6207,38.6709,11.7,124.0,2022-11-02 20:40:48.071161,2022-11-02 11:38:13,2022-11-02 08:14:16,under way using engine,915.807411,1573.536915,2022-11-04 23:13:20.035036,2022-11-05 01:34:31,0 days 02:21:10.964964
25,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6058,38.6752,12.0,101.0,2022-11-02 20:35:45.203472,2022-11-02 11:34:23,2022-11-02 08:14:16,under way using engine,917.183776,1589.732763,2022-11-04 23:09:30.035036,2022-11-05 01:34:31,0 days 02:25:00.964964
85,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.1918,35.4455,12.5,147.0,2022-11-04 03:33:41.281890,2022-11-03 18:32:16,2022-11-02 07:44:12,under way using engine,333.787423,588.528544,2022-11-04 22:58:44.496168,2022-11-05 01:34:31,0 days 02:35:46.503832


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [129]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [130]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_seg_{the_number_of_segment}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [131]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 02:41:00.964964
평균: 0 days 00:51:27.068309322


In [132]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
237,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6501,34.1865,6.9,68.0,2022-11-04 23:24:53.599991,2022-11-04 14:24:23,2022-11-04 01:38:11,under way using engine,137.996749,261.750760,2022-11-05 01:34:29.505037,2022-11-05 01:34:31,0 days 00:00:01.494963
183,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.0599,34.0799,6.5,79.0,2022-11-04 18:52:18.020141,2022-11-04 09:48:24,2022-11-02 07:44:12,under way using engine,189.127328,372.171594,2022-11-05 01:34:40.419380,2022-11-05 01:34:31,0 days 00:00:09.419380
211,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.3726,34.1093,6.5,66.0,2022-11-04 21:13:37.768693,2022-11-04 12:13:03,2022-11-02 07:44:12,under way using engine,163.440426,321.623979,2022-11-05 01:34:05.289072,2022-11-05 01:34:31,0 days 00:00:25.710928
220,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.4674,34.1327,6.4,63.0,2022-11-04 21:59:03.571546,2022-11-04 12:58:23,2022-11-02 07:44:12,under way using engine,154.892014,307.900210,2022-11-05 01:35:15.728795,2022-11-05 01:34:31,0 days 00:00:44.728795
264,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9472,34.2824,7.2,59.0,2022-11-05 01:41:07.975263,2022-11-04 16:40:24,2022-11-04 04:32:12,under way using engine,110.492280,204.631195,2022-11-05 01:35:25.121648,2022-11-05 01:34:31,0 days 00:00:54.121648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,122.7751,38.0019,11.1,97.0,2022-11-03 12:14:32.908857,2022-11-03 02:58:16,2022-11-02 07:44:12,under way using engine,636.722525,1079.193658,2022-11-05 03:55:30.660447,2022-11-05 01:34:31,0 days 02:20:59.660447
26,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6207,38.6709,11.7,124.0,2022-11-02 20:40:48.071161,2022-11-02 11:38:13,2022-11-02 08:14:16,under way using engine,915.807411,1573.536915,2022-11-04 23:13:20.035036,2022-11-05 01:34:31,0 days 02:21:10.964964
25,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6058,38.6752,12.0,101.0,2022-11-02 20:35:45.203472,2022-11-02 11:34:23,2022-11-02 08:14:16,under way using engine,917.183776,1589.732763,2022-11-04 23:09:30.035036,2022-11-05 01:34:31,0 days 02:25:00.964964
85,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.1918,35.4455,12.5,147.0,2022-11-04 03:33:41.281890,2022-11-03 18:32:16,2022-11-02 07:44:12,under way using engine,333.787423,588.528544,2022-11-04 22:58:44.496168,2022-11-05 01:34:31,0 days 02:35:46.503832


# ETA 실험 (interpolation 10000), AKA BHUM 7항차 저속

In [133]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.669615e+09,117.794500,38.979000,7.700000
1,1.669615e+09,117.795166,38.978525,7.779298
2,1.669615e+09,117.795832,38.978050,7.858596
3,1.669616e+09,117.796498,38.977575,7.937894
4,1.669616e+09,117.797164,38.977100,8.017193
...,...,...,...,...
9995,1.669881e+09,128.911219,34.852268,8.176007
9996,1.669881e+09,128.911352,34.853251,8.157005
9997,1.669881e+09,128.911484,34.854234,8.138003
9998,1.669882e+09,128.911617,34.855217,8.119002


In [134]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment 120

In [135]:
the_number_of_segment = 120

In [136]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [137]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [138]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

9.884330952551009
9.47866109389574
13.054702692884405
14.462760822080215
11.218944942527177
11.063561625483
11.0967509366992
11.258421743619067
11.229672477365725
11.190177210015836
11.18807424290425
11.196652617699886
11.007780253954888
10.818936779204309
10.753038925005503
10.80497141921395
10.885170651652471
10.991535054399597
10.946145560953457
10.74761120951379
10.548270889768958
10.965396409969243
11.14327392008918
11.349413819220397
11.364824718972605
11.044791661334486
10.819647557340767
10.657962389403677
10.604455676835087
10.609711887681613
10.614968098528003
10.62022430937453
10.625480520221215
10.632772390728869
10.662870856265101
10.699337891493682
10.735804926722185
10.772271961950608
10.808738997179267
10.832937746795075
10.816690914063408
10.797261424529177
10.777831934994946
10.758402445460716
10.738972955926487
10.719543466392258
10.700113976858027
10.680684487323795
10.661254997789566
10.641825508255334
10.622396018721101
10.60296652918685
10.583537039652617
10.5641

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [139]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [140]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_seg_{the_number_of_segment}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [141]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 03:05:11.696930
평균: 0 days 01:02:36.843147084


In [142]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
261,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9146,34.2690,7.3,59.0,2022-11-05 01:25:59.438198,2022-11-04 16:25:43,2022-11-04 04:14:10,under way using engine,113.686369,209.015615,2022-11-05 01:34:38.710915,2022-11-05 01:34:31,0 days 00:00:07.710915
333,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,128.8314,34.6292,7.0,42.0,2022-11-05 08:55:11.127437,2022-11-04 23:51:52,2022-11-04 11:26:10,under way using engine,37.866529,71.230699,2022-11-05 01:36:01.208738,2022-11-05 01:34:31,0 days 00:01:30.208738
338,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,128.8831,34.6755,7.1,40.0,2022-11-05 09:25:27.895215,2022-11-05 00:24:02,2022-11-05 12:20:10,under way using engine,33.214596,61.984317,2022-11-05 01:32:58.658989,2022-11-05 01:34:31,0 days 00:01:32.341011
75,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.4065,38.3485,10.5,105.0,2022-11-03 04:20:06.175754,2022-11-02 19:17:58,2022-11-02 07:08:11,under way using engine,760.164317,1278.266010,2022-11-05 01:36:17.453305,2022-11-05 01:34:31,0 days 00:01:46.453305
64,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.0239,38.5067,11.4,114.0,2022-11-03 02:29:03.864811,2022-11-02 17:28:44,2022-11-02 05:14:14,under way using engine,797.848008,1360.631328,2022-11-05 01:36:23.585081,2022-11-05 01:34:31,0 days 00:01:52.585081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6912,38.6548,11.5,99.0,2022-11-02 21:00:59.275952,2022-11-02 11:55:53,2022-11-02 08:14:16,under way using engine,909.486087,1554.668777,2022-11-04 23:06:49.303070,2022-11-05 01:34:31,0 days 02:27:41.696930
27,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6690,38.6594,11.8,105.0,2022-11-02 20:50:54.269771,2022-11-02 11:50:23,2022-11-02 08:14:16,under way using engine,911.455420,1570.416337,2022-11-04 23:01:19.303070,2022-11-05 01:34:31,0 days 02:33:11.696930
26,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6207,38.6709,11.7,124.0,2022-11-02 20:40:48.071161,2022-11-02 11:38:13,2022-11-02 08:14:16,under way using engine,915.807411,1573.536915,2022-11-04 22:49:09.303070,2022-11-05 01:34:31,0 days 02:45:21.696930
25,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6058,38.6752,12.0,101.0,2022-11-02 20:35:45.203472,2022-11-02 11:34:23,2022-11-02 08:14:16,under way using engine,917.183776,1589.732763,2022-11-04 22:45:19.303070,2022-11-05 01:34:31,0 days 02:49:11.696930


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [143]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [144]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_seg_{the_number_of_segment}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [145]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 03:05:11.696930
평균: 0 days 00:48:58.551619516


In [146]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
261,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9146,34.2690,7.3,59.0,2022-11-05 01:25:59.438198,2022-11-04 16:25:43,2022-11-04 04:14:10,under way using engine,113.686369,209.015615,2022-11-05 01:34:38.710915,2022-11-05 01:34:31,0 days 00:00:07.710915
181,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.0455,34.0785,6.5,79.0,2022-11-04 18:42:12.744927,2022-11-04 09:41:44,2022-11-02 07:44:12,under way using engine,190.334991,374.548078,2022-11-05 01:34:10.932917,2022-11-05 01:34:31,0 days 00:00:20.067083
185,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.0893,34.0824,6.6,80.0,2022-11-04 19:02:23.752887,2022-11-04 10:01:44,2022-11-02 07:44:12,under way using engine,186.688223,363.811694,2022-11-05 01:34:59.578079,2022-11-05 01:34:31,0 days 00:00:28.578079
190,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.1441,34.0899,6.4,76.0,2022-11-04 19:27:38.264334,2022-11-04 10:27:14,2022-11-02 07:44:12,under way using engine,181.990342,361.767292,2022-11-05 01:34:01.593081,2022-11-05 01:34:31,0 days 00:00:29.406919
231,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.5846,34.1687,6.7,61.0,2022-11-04 22:54:34.372407,2022-11-04 13:54:14,2022-11-04 01:38:11,under way using engine,143.924624,277.859600,2022-11-05 01:35:09.936052,2022-11-05 01:34:31,0 days 00:00:38.936052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6912,38.6548,11.5,99.0,2022-11-02 21:00:59.275952,2022-11-02 11:55:53,2022-11-02 08:14:16,under way using engine,909.486087,1554.668777,2022-11-04 23:06:49.303070,2022-11-05 01:34:31,0 days 02:27:41.696930
27,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6690,38.6594,11.8,105.0,2022-11-02 20:50:54.269771,2022-11-02 11:50:23,2022-11-02 08:14:16,under way using engine,911.455420,1570.416337,2022-11-04 23:01:19.303070,2022-11-05 01:34:31,0 days 02:33:11.696930
26,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6207,38.6709,11.7,124.0,2022-11-02 20:40:48.071161,2022-11-02 11:38:13,2022-11-02 08:14:16,under way using engine,915.807411,1573.536915,2022-11-04 22:49:09.303070,2022-11-05 01:34:31,0 days 02:45:21.696930
25,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.6058,38.6752,12.0,101.0,2022-11-02 20:35:45.203472,2022-11-02 11:34:23,2022-11-02 08:14:16,under way using engine,917.183776,1589.732763,2022-11-04 22:45:19.303070,2022-11-05 01:34:31,0 days 02:49:11.696930


# 결과 확인

In [147]:
result_pool = pd.DataFrame(result_df_dict)
result_pool

,experiment_no,mean_speed,max_speed
0,4_1_2_3_seg_20,0 days 01:38:32.684934629,0 days 03:58:42.341169
1,4_2_1_3_seg_20,0 days 01:36:43.464540446,0 days 04:01:07.590395
2,4_1_2_3_seg_30,0 days 01:26:21.585082521,0 days 04:01:30.675181
3,4_2_1_3_seg_30,0 days 01:17:35.111568161,0 days 04:01:30.675181
4,4_1_2_3_seg_40,0 days 01:16:38.288633061,0 days 02:48:36.719958
5,4_2_1_3_seg_40,0 days 01:07:01.690251693,0 days 02:48:36.719958
6,4_1_2_3_seg_40,0 days 01:16:38.288633061,0 days 02:48:36.719958
7,4_2_1_3_seg_40,0 days 01:07:01.690251693,0 days 02:48:36.719958
8,4_1_2_3_seg_50,0 days 01:14:10.445769244,0 days 03:13:24.529074
9,4_2_1_3_seg_50,0 days 01:01:26.578008708,0 days 03:13:24.529074


In [148]:
result_pool.sort_values(by="mean_speed")

,experiment_no,mean_speed,max_speed
17,4_2_1_3_seg_120,0 days 00:48:58.551619516,0 days 03:05:11.696930
11,4_2_1_3_seg_75,0 days 00:51:20.246042787,0 days 02:53:11.846677
15,4_2_1_3_seg_100,0 days 00:51:27.068309322,0 days 02:41:00.964964
13,4_2_1_3_seg_90,0 days 00:54:49.992278455,0 days 03:11:12.976273
9,4_2_1_3_seg_50,0 days 01:01:26.578008708,0 days 03:13:24.529074
16,4_1_2_3_seg_120,0 days 01:02:36.843147084,0 days 03:05:11.696930
10,4_1_2_3_seg_75,0 days 01:03:25.412288478,0 days 02:53:11.846677
14,4_1_2_3_seg_100,0 days 01:05:32.943752793,0 days 02:41:00.964964
5,4_2_1_3_seg_40,0 days 01:07:01.690251693,0 days 02:48:36.719958
7,4_2_1_3_seg_40,0 days 01:07:01.690251693,0 days 02:48:36.719958


In [149]:
for i in globals():
    if 'exp_result_df_' in i:
        globals()[i]

In [150]:
# exp_result_df_4_1_2_3

In [151]:
# result_pool.to_csv(f"230126_result_pool_{interpolation_number}.csv", encoding="utf-8 sig", header=True, index=False)